In [ ]:
!git config --global user.email 'gmsltjq123@naver.com'
!git config --global user.name 'user'

In [20]:
ls

Infinite_Buying_Method_Backtest  README.md


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
def round_half_up_to_two(num):
    if pd.isna(num):  # NaN 값 처리
        return num

    num = num * 100  # 소수점 둘째 자리로 변환
    if num - int(num) >= 0.5:
        return (int(num) + 1) / 100  # 둘째 자리에서 반올림 후 다시 나누기
    else:
        return int(num) / 100  # 둘째 자리에서 반올림 후 다시 나누기

In [ ]:
def get_data(ticker, start, end):
    start_date = start
    end_date = end

    df = yf.download(ticker, start=start_date, end=end_date)
    df = df.drop(columns = ['Volume', 'Adj Close']) # 시가, 고가, 저가, 종 데이터만 유지
    df.index = df.index.date
    df['Open'] = df['Open'].apply(round_half_up_to_two) # 호가 단위 0.01$
    df['High'] = df['High'].apply(round_half_up_to_two)
    df['Low'] = df['Low'].apply(round_half_up_to_two)
    df['Close'] = df['Close'].apply(round_half_up_to_two)
    df['Return'] = (df['Close'].pct_change() * 100).apply(round_half_up_to_two) # 전일 대비 등락율 계산 (% 단위)

    return df

In [ ]:
def buy_shares(funds, price, one_buy_amount, qty, avg_price, holdings):
    qty = one_buy_amount // price # 주문 수량 : 일일 총액의 절반까지 구매 가능, 정수로 주문
    holdings += qty # 보유 주식 수
    funds -= price * qty # 주문 성공한 만큼 자금 차감
    avg_price = ((avg_price * (holdings - qty)) + (price * qty)) / holdings if holdings > 0 else 0
    return funds, avg_price, holdings, qty

In [ ]:
def sell_shares(funds, price, qty_sell, holdings):
    holdings -= qty_sell
    funds += qty_sell * price
    return funds, holdings

In [ ]:
def quarter_sell_strategy(df, df_res, i, funds, price, avg_price, highest_price, one_buy_amount, holdings, quarter_sell, quarter_count):
    df_res.loc[i, '쿼터 손절'] = 'O'
    quarter_count += 1

    if quarter_count % 10 == 1: # 쿼터 매도 회차 중 최초 한번만 one_buy_amount 계산하고, 이후 10회차까지 매도에 실패했다면, one_buy_amount를 다시 계산
        qty_sell = holdings // 4
        df_res.loc[i, 'MOC 매도'] = qty_sell
        funds, holdings = sell_shares(funds, price, qty_sell, holdings) # 25% 분량을 MOC 매도
        one_buy_amount = funds // 10 # 1회 총액 : 기존 수익과 합산하여 10분할

    else:
      ##<------------------------ 매수 전략 ------------------------>##
      # 평단 - 10% 가격에 LOC 매수
      if price < (avg_price * 0.9):
          funds, avg_price, holdings, qty = buy_shares(funds, price, one_buy_amount, holdings, avg_price, holdings)

      ##<------------------------ 매도 전략 ------------------------>##
      # 보유 수량의 75%: 평단 + 10% 가격에 지정가 매도
      if highest_price >= (avg_price * 1.1):
          qty_sell = holdings * 3 // 4
          funds, holdings = sell_shares(funds, price, qty_sell, holdings)
          quarter_sell = False # 매도 체결 시 쿼터 손절 탈출

      # 나머지 25%: 평단 - 10% 가격에 매도
      if price >= (avg_price * 0.9):
          qty_sell = holdings // 4
          funds, holdings = sell_shares(funds, price, qty_sell, holdings)
          df_res.loc[i, '-10% LOC 매도'] = qty_sell
          quarter_sell = False # 매도 체결 시 쿼터 손절 탈출

    return funds, avg_price, holdings, quarter_sell, quarter_count

In [ ]:
def normal_buy_strategy(df, df_res, i, funds, price, avg_price, one_buy_amount, holdings, T, is_first_period):
    star_percent_price = round_half_up_to_two(avg_price * (1 + (10 - T / 2) / 100))
    df_res.loc[i, '쿼터 손절'] = 'X'

    if is_first_period:
        funds, avg_price, holdings, qty = buy_shares(funds, price, one_buy_amount, 0, avg_price, holdings)
        df_res.loc[i, 'LOC 평단 매수'] = qty
        is_first_period = False

    else: # 첫 회차가 아닌 모든 경우
      ##<------------------------ 매수 전략 ------------------------>##
      if T < 20: # 한 회차 금액의 50%: 평단가에 LOC 매수 / 나머지 50%: 별% 가격에 LOC 매수
          if price <= avg_price:
              funds, avg_price, holdings, qty = buy_shares(funds, price, one_buy_amount / 2, holdings, avg_price, holdings)
              df_res.loc[i, 'LOC 평단 매수'] = qty

          if price < star_percent_price: # 별% 매수는 별% - 0.01이므로 등호 X
              funds, avg_price, holdings, qty = buy_shares(funds, price, one_buy_amount / 2, holdings, avg_price, holdings)
              df_res.loc[i, 'LOC 큰 매수'] = qty

      elif T >= 20: # 별% 가격에 LOC 매수
          if price < star_percent_price:
              funds, avg_price, holdings, qty = buy_shares(funds, price, one_buy_amount, holdings, avg_price, holdings)
              df_res.loc[i, 'LOC 큰 매수'] = qty

    df_res.loc[i, '평균단가'] = round_half_up_to_two(avg_price)

    return funds, avg_price, holdings, is_first_period

In [ ]:
def normal_sell_strategy(df, df_res, i, price, avg_price, highest_price, holdings, funds, star_percent_price, is_first_period):
    # 매도 전략 (보유 수량의 75%: 평단 + 10% 가격에 지정가 매도)
    holdings_75  = holdings * 3 // 4
    holdings_25  = holdings // 4
    existing_holdings = holdings # holdings의 변경 여부 판단

    if highest_price >= (avg_price * 1.1): # 지정가 매도이므로, price가 아닌 highest_price와 비교
        if holdings == 1:
           qty_sell = 1 # 보유 주식 수가 1개인 경우 + 10% 가격에 지정가 매도
        else:
           qty_sell = holdings_75
        funds, holdings = sell_shares(funds, price, qty_sell, holdings)
        df_res.loc[i, '10% 지정가 매도'] = qty_sell

    # 매도 전략 (나머지 25%: 별% 가격에 LOC 매도)
    if price >= star_percent_price:
        qty_sell = holdings_25
        funds, holdings = sell_shares(funds, price, qty_sell, holdings)
        df_res.loc[i, 'LOC 별% 매도'] = qty_sell

    if existing_holdings != holdings and holdings == 0: # holidings가 변화했으면서, 보유 수량이 0이 된 경우
      is_first_period = True # 첫회차부터 다시 반복

    return funds, holdings, is_first_period

In [ ]:
# 무한 매수법 시뮬레이션 함수
def infinite_buy_simulation(df, df_res, initial_funds, buy_portion, start_idx, simulation_period):
    funds = initial_funds
    one_buy_amount = initial_funds / buy_portion # 한 회차 매수액
    holdings = 0 # 보유 주식 수
    avg_price = 0 # 평단가
    quarter_sell = False
    quarter_count = 0 # 쿼터 매도 회차
    is_first_period = True # 첫 회차 매수

    for i in range(start_idx, start_idx + simulation_period + 1): # start ~ end까지의 시뮬레이션 진행
        df_res.loc[i, '날짜'] = df.index[i]
        df_res.loc[i, '시가'] = df['Open'].iloc[i]
        df_res.loc[i, '고가'] = df['High'].iloc[i]
        df_res.loc[i, '종가'] = df['Close'].iloc[i]
        df_res.loc[i, '등락율'] = f"{round_half_up_to_two(df['Return'].iloc[i])}%"
        price = df['Close'].iloc[i]
        highest_price = df['High'].iloc[i] # 고가
        T = round_half_up_to_two((holdings * avg_price) / one_buy_amount)
        df_res.loc[i, '별% 값'] = f"{round_half_up_to_two(10 - T / 2)}%"


        if 39 < T <= 40:
            quarter_sell = True

        if quarter_sell: # 쿼터 손절 모드
            df_res.loc[i, '별% 가격'] = 'Invalid' # 쿼터 매도에서는 별% 가격을 사용하지 않음
            funds, avg_price, holdings, quarter_sell, quarter_count = quarter_sell_strategy(
                df, df_res, i, funds, price, avg_price, highest_price, one_buy_amount, holdings, quarter_sell, quarter_count)
        else:
            # normal_buy_strategy에 의해 늘어난 holdings가 normal_sell_strategy의 holdings에 영향을 미치면 안됨. (LOC 매수이므로)
            funds, avg_price, new_buy_holdings, is_first_period = normal_buy_strategy(df, df_res, i, funds, price, avg_price, one_buy_amount, holdings, T, is_first_period)
            star_percent_price = round_half_up_to_two(avg_price * (1 + (10 - T / 2) / 100))
            df_res.loc[i, '별% 가격'] = star_percent_price
            funds, new_sell_holdings, is_first_period = normal_sell_strategy(df, df_res, i, price, avg_price, highest_price, holdings, funds, star_percent_price, is_first_period)

            # holdings 업데이트
            increment = new_buy_holdings - holdings # 새로 산 주식 수
            decrement = holdings - new_sell_holdings # 판매한 주식 수
            holdings = holdings + increment - decrement

        df_res.loc[i, '보유 주식 수'] = holdings
        df_res.loc[i, '예수금'] = funds
        df_res.loc[i, '총 평가액'] = funds + (price * holdings)
        df_res.loc[i, '수익율(%)'] = "0.0%" if holdings == 0 else f"{round_half_up_to_two((price * holdings - avg_price * holdings) / (avg_price * holdings) * 100)}%"


    final_value = funds + (holdings * df['Close'].iloc[start_idx + simulation_period])

    return round_half_up_to_two((final_value / initial_funds - 1) * 100), df_res, final_value


In [ ]:
from datetime import datetime, timedelta
from IPython.display import display

# 시뮬레이션 시작일과 종료일 설정
start_date = '2020-01-01'
end_date = '2022-01-31'

# 초기 자본금, 분할 횟수 설정
initial_funds = 40000
buy_portion = 40 # 40분할 적용

end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')
next_day = end_date_dt + timedelta(days=1) # 하루 다음 날로 변환
end_day_next = next_day.strftime('%Y-%m-%d') # 다시 문자열로 변환

df = get_data(ticker='TQQQ', start=start_date, end=end_day_next) # end 날짜는 포함되지 않으므로, 종료일의 다음날을 전달
df_res = pd.DataFrame(columns = ['날짜', '시가', '고가', '종가', '등락율', '별% 값', '별% 가격', 'LOC 평단 매수', 'LOC 큰 매수', '10% 지정가 매도', 'LOC 별% 매도', 'MOC 매도', '-10% LOC 매도', '보유 주식 수', '쿼터 손절', '평균단가', '예수금', '총 평가액', '수익율(%)'])

# 1회차 매수액을 $1000로 가정
return_rate, df_res, final_value = infinite_buy_simulation(df, df_res, initial_funds=initial_funds, buy_portion=buy_portion, start_idx=0, simulation_period=len(df)-1)

# 결측 값 대체
df_res.loc[0, '등락율'] = 'Irrelavant' # 첫째날의 등락률을 표기하지 않음.
df_res.loc[0, '별% 가격'] = 'Invalid'
df_res = df_res.fillna(0)

# 실수를 정수형으로 변환
df_res['LOC 평단 매수'] = df_res['LOC 평단 매수'].fillna(0).astype(int)
df_res['LOC 큰 매수'] = df_res['LOC 큰 매수'].fillna(0).astype(int)
df_res['10% 지정가 매도'] = df_res['10% 지정가 매도'].fillna(0).astype(int)
df_res['LOC 별% 매도'] = df_res['LOC 별% 매도'].fillna(0).astype(int)
df_res['MOC 매도'] = df_res['MOC 매도'].fillna(0).astype(int)
df_res['-10% LOC 매도'] = df_res['-10% LOC 매도'].fillna(0).astype(int)
df_res['보유 주식 수'] = df_res['보유 주식 수'].fillna(0).astype(int)

# 중앙 정렬하여 출력
df_res_style = df_res.style.set_properties(**{'text-align': 'center'})  # 중앙 정렬 설정
display(df_res_style)  # DataFrame 출력

print()
print('<------------------ ' + start_date + " ~ " + end_date + ' 동안의 자산 변동 결과 ------------------>')
print('최초 보유 금액:', end=' ')
print(initial_funds)
print('최종 보유 금액:', end=' ')
print(final_value)
print('원금 변화율:', end=' ')
print(str(return_rate) + '%')
print('<----------------------------------------------------------------------------------->')


[*********************100%***********************]  1 of 1 completed



<------------------ 2020-01-01 ~ 2022-01-31 동안의 자산 변동 결과 ------------------>
최초 보유 금액: 40000
최종 보유 금액: 60252.47000000011
원금 변화율: 50.63%
<----------------------------------------------------------------------------------->
